In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["OMP_NUM_THREADS"] = "12"

import torch

In [2]:

import time
import yaml
import pandas as pd
from rdkit import Chem

In [10]:
from rassp.msutil.masscompute import FragmentFormulaPeakEnumerator

valid_atoms = {1, 5, 6, 7, 8, 9, 14, 15, 16, 17, 35, 53}
#valid_atoms = {1, 6, 7, 8, 9, 15, 16, 17}
num_peaks_per_formula = 12
#max_formulae = 100000
#max_formulae = 4096
max_formulae = 16384

ffe = FragmentFormulaPeakEnumerator(sorted(valid_atoms), use_highres=True, max_peak_num=num_peaks_per_formula)

def goodmol(mol):
        if len(mol.GetAtoms()) > 48:
            return False
        
        atoms = { a.GetAtomicNum() for a in mol.GetAtoms() }
        if not atoms < valid_atoms:
            return False
            
        f,m = ffe.get_frag_formulae(mol)
        if len(f) > max_formulae:
            return False

        return True


In [4]:
test = pd.read_parquet('test.pq')
test

,rdmol,spect,smiles
8,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[37.0, 14.99], [38.0, 57.95], [39.0, 333.7], ...",CC1=NN(C(=O)C1(C)C)C2=CC=CC=C2
19,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[15.0, 4.0], [26.0, 6.99], [27.0, 99.91], [28...",CCC(C(C)CC(C)C(=O)CC)O
20,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[25.0, 7.99], [26.0, 88.92], [27.0, 146.87], ...",CC=NC(CC#N)C#N
21,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[27.0, 0.5], [28.0, 0.5], [29.0, 2.0], [30.0,...",CCN(CC)CCNC(=O)C1=CC(=C(C=C1OC)N)Cl
26,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[24.0, 1.0], [25.0, 4.0], [26.0, 30.97], [27....",C1=CC(=NC(=C1)C=O)C=O
...,...,...,...
197094,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[14.0, 1.0], [15.0, 12.99], [16.0, 2.0], [17....",COC1=CC=C(C=C1)C2(CC2)C(=O)O
197096,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[12.0, 2.0], [13.0, 3.0], [14.0, 5.99], [15.0...",C[Si](Cl)Cl
197099,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[41.0, 11.99], [43.0, 5.99], [44.0, 21.98], [...",C1=CC=C2C(=C1)C(=O)C(=C(C2=O)Cl)NC3=CC=C(C=C3)N
197102,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[36.0, 0.5], [37.0, 4.0], [39.0, 73.93], [41....",CC1=CC(=C(C(=C1)C)S(=O)(=O)NCCC2=NC3=CC=CC=C3N2)C


In [11]:
mediumtest = test[[ goodmol(Chem.AddHs(Chem.Mol(m))) for m in test.rdmol ]]
mediumtest

,rdmol,spect,smiles
8,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[37.0, 14.99], [38.0, 57.95], [39.0, 333.7], ...",CC1=NN(C(=O)C1(C)C)C2=CC=CC=C2
19,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[15.0, 4.0], [26.0, 6.99], [27.0, 99.91], [28...",CCC(C(C)CC(C)C(=O)CC)O
20,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[25.0, 7.99], [26.0, 88.92], [27.0, 146.87], ...",CC=NC(CC#N)C#N
21,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[27.0, 0.5], [28.0, 0.5], [29.0, 2.0], [30.0,...",CCN(CC)CCNC(=O)C1=CC(=C(C=C1OC)N)Cl
26,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[24.0, 1.0], [25.0, 4.0], [26.0, 30.97], [27....",C1=CC(=NC(=C1)C=O)C=O
...,...,...,...
197094,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[14.0, 1.0], [15.0, 12.99], [16.0, 2.0], [17....",COC1=CC=C(C=C1)C2(CC2)C(=O)O
197096,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[12.0, 2.0], [13.0, 3.0], [14.0, 5.99], [15.0...",C[Si](Cl)Cl
197099,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[41.0, 11.99], [43.0, 5.99], [44.0, 21.98], [...",C1=CC=C2C(=C1)C(=O)C(=C(C2=O)Cl)NC3=CC=C(C=C3)N
197102,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[36.0, 0.5], [37.0, 4.0], [39.0, 73.93], [41....",CC1=CC(=C(C(=C1)C)S(=O)(=O)NCCC2=NC3=CC=CC=C3N2)C


In [12]:
mediumtest.to_parquet('mediumtest.pq')

In [13]:
train = pd.read_parquet('train.pq')
train

,rdmol,spect,smiles
0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[12.0, 1.0], [13.0, 4.0], [14.0, 25.98], [15....",COC1=CC=C(C=C1)C(=O)CBr
1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[27.0, 37.97], [28.0, 8.99], [29.0, 2.0], [30...",CCCC(=O)NC1=CC=CC=C1
2,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[41.0, 370.67], [42.0, 999.0], [43.0, 247.78]...",CC(CC#N)N1CC1
3,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[41.0, 181.84], [42.0, 105.9], [43.0, 251.77]...",C1=CC2=C(NC3=C(C=C(C=C3)CN)C(=O)N2)N=C1
4,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[35.0, 2.0], [36.0, 3.0], [37.0, 5.99], [38.0...",CC1=NN=C(O1)CCC(=O)O
...,...,...,...
197106,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[27.0, 105.9], [28.0, 32.97], [30.0, 16.98], ...",C1CC(C(C1)Br)F
197107,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[51.0, 134.88], [55.0, 33.97], [65.0, 38.96],...",CC(C)NC=CC1=CC(=C(C=C1)O)O
197108,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[41.0, 9.99], [42.0, 29.97], [43.0, 9.99], [4...",CN(C)C(=O)C1=CC=CC2=CC=CC=C21
197110,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[11.0, 0.5], [12.0, 0.5], [13.0, 1.0], [14.0,...",CC1=CC=C(S1)C


In [14]:
mediumtrain = train[[ goodmol(Chem.AddHs(Chem.Mol(m))) for m in train.rdmol ]]
mediumtrain

,rdmol,spect,smiles
0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[12.0, 1.0], [13.0, 4.0], [14.0, 25.98], [15....",COC1=CC=C(C=C1)C(=O)CBr
1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[27.0, 37.97], [28.0, 8.99], [29.0, 2.0], [30...",CCCC(=O)NC1=CC=CC=C1
2,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[41.0, 370.67], [42.0, 999.0], [43.0, 247.78]...",CC(CC#N)N1CC1
3,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[41.0, 181.84], [42.0, 105.9], [43.0, 251.77]...",C1=CC2=C(NC3=C(C=C(C=C3)CN)C(=O)N2)N=C1
4,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[35.0, 2.0], [36.0, 3.0], [37.0, 5.99], [38.0...",CC1=NN=C(O1)CCC(=O)O
...,...,...,...
197106,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[27.0, 105.9], [28.0, 32.97], [30.0, 16.98], ...",C1CC(C(C1)Br)F
197107,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[51.0, 134.88], [55.0, 33.97], [65.0, 38.96],...",CC(C)NC=CC1=CC(=C(C=C1)O)O
197108,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[41.0, 9.99], [42.0, 29.97], [43.0, 9.99], [4...",CN(C)C(=O)C1=CC=CC2=CC=CC=C21
197110,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[11.0, 0.5], [12.0, 0.5], [13.0, 1.0], [14.0,...",CC1=CC=C(S1)C


In [15]:
mediumtrain.to_parquet('mediumtrain.pq')

In [16]:
len(train) + len(test), len(mediumtrain) + len(mediumtest)

(197112, 194008)

In [17]:
import sys
sys.path.append('./rassp')
from forward_train import train as rassp_train


In [18]:
with open('rassp/expconfig/demo.yaml') as cf:
    exp_config = yaml.load(cf,Loader=yaml.FullLoader)


In [19]:
exp_config['max_epochs'] = 3000
exp_config['epoch_size'] = 4096
exp_config['seed'] = int(time.time())
exp_config['batch_size'] = 2
exp_config['checkpoint_every_n_epochs'] = 1
exp_config['attempt_load_recent_checkpoint'] = True
exp_config['validate_every'] = 10
exp_config['cluster_config']['data_dir'] = '.'
exp_config['DATALOADER_NUM_WORKERS'] = 2

exp_config['featurize_config']['explicit_formulae_config']['max_formulae'] = max_formulae
exp_config['featurize_config']['element_oh']=list(valid_atoms)
exp_config['featurize_config']['explicit_formulae_config']['formula_possible_atomicno'] = list(valid_atoms)
exp_config['net_params']['spect_out_config']['formula_oh_sizes'] = [50, 46, 30, 30, 30, 30, 30, 30, 20, 20, 16, 8] # XXX: guess



exp_config['exp_data']['data'][0]['db_filename'] = 'mediumtrain.pq'
exp_config['exp_data']['data'][1]['db_filename'] = 'mediumtest.pq'

with open('exp.yml','w') as y:
    yaml.dump(exp_config,y)


In [23]:
# %%capture trainout
# just test, run as train_medium.py seriously
rassp_train('medium',exp_config,exp_config_filename='exp.yml',USE_CUDA=True)

RuntimeError: received 0 items of ancdata

In [ ]:
trainout.show()

using checkpoint_dir: checkpoints
we are training with 155254
we are testing with 38754
it took 3.1s to load the data
{'init_noise': 0.01, 'init_bias': 0.0, 'resnet': True, 'int_d': 512, 'layer_n': 16, 'agg_func': 'goodmax', 'gml_class': 'GraphMatLayersNormAfterRes', 'gml_config': {'layer_class': 'GraphMatLayerFast3', 'layer_config': {'dropout': 0.0, 'nonlin': 'leakyrelu', 'swap_init': False}}, 'g_feature_n': 45, 'inner_norm': 'layer', 'input_norm': 'batch', 'spect_out_class': 'MolAttentionGRU', 'spect_out_config': {'formula_oh_sizes': [50, 46, 30, 30, 30, 30, 30, 30, 20, 20, 16, 8], 'formula_oh_accum': True, 'internal_d': 128, 'embedding_key_size': 8, 'gru_layer_n': 3, 'linear_layer_n': 9}, 'GS': 4, 'spect_bin': <rassp.msutil.binutils.SpectrumBins object at 0x7fcac1be1d00>}
GCN module had 15869952 nparams
Spectral module had 4157745 nparams
Net overall had 20027787 nparams
looking to load a recent checkpoint...
NOT LOADING FROM ANY CHECKPOINT
NOT USING CUSTOM COLLATE_FN
gml.gl.0.weigh